# PL/pgSQL

PL/pgSQL 是一种过程语言（procedural language）。它是 PostgreSQL 的一种扩展，允许用户编写存储过程、触发器、函数等。包括：控制结构、循环和复杂的计算。

PL/pgSQL 的设计目的是：
- 创建用户自定义的函数，存储过程和触发器。
- 通过增加控制结构，例如条件和循环，扩展标注 SQL。
- 继承所有用户定义的函数、运算符和类型。

优点：
PostgreSQL 只能单独执行语句，对于多个语句来说，它需要在客户端执行多次，而 PL/pgSQL 可以将多个语句封装在一个函数中，然后一次性执行，减少了客户端和服务器之间的通信次数，提高了效率。 封装的函数可以存储在服务中，可以被多个客户端调用，提高了代码的复用性。这样做到了：
- 减少应用程序和 PostgreSQL 数据库服务器之间的往返次数
- 避免在应用程序和服务器之间传输即时结果

缺点：
- 降低开发效率。因为 PL/pgSQL 语法比较复杂，需要花费更多的时间来学习；
- PL/pgSQL 的代码版本难以管理且难以调试；
- 降低可移植性。PL/pgSQL 是 PostgreSQL 的专有语言，不同的数据库系统不支持 PL/pgSQL，所以如果需要迁移数据库，需要重写 PL/pgSQL 代码。


## `$$` 字符串常量

PL/pgSQL 代码块使用 `$$` 字符串常量来定义。`$$` 之间的内容是 PL/pgSQL 代码块的内容。`$$` 之间的内容可以包含任何有效的 SQL 语句。在一般的 `select` 语句中，如果遇到了单引号（'）或者双引号（"），需要使用转义字符（\）来转义，但是在 `$$` 之间的内容中，不需要转义。

select 写法：
> select 'I''m a string constant';

$$ 写法：
> select $$I'm a string constant$$;


$$ 语法
```sql
$tag$<string_constant>$tag$
```
上述语法中，`tag` 是可选的。它遵循与不带引号的标识符相同的规则：
- 必须以字母或下划线开头；
- 后续字符可以是字母、数字或下划线；
- 不能包含空格或其他保留的特殊字符；
- 长度限制在 63 个字符以内。

In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


## 在匿名块中使用 $$ 字符串常量

In [2]:
# 一般写法
sql = """
do 
'declare
   film_count integer;
begin 
   select count(*) into film_count
   from film;

   raise notice ''The number of films: %'', film_count;
end;'
;
"""

cursor.execute(sql)

The number of films: 1000


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x21db88b56f0>

In [3]:
# $$ 写法
sql = """
do 
$$
declare
   film_count integer;
begin 
   select count(*) into film_count
   from film;
   raise notice 'The number of films: %', film_count;
end;
$$;
"""
cursor.execute(sql)

The number of films: 1000


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x21db88b56f0>

## 在函数中使用 $$ 字符串常量

```sql
create function function_name(param_list) 
    returns datatype
language lang_name
as 
 'function_body'
```

`function_body` 是一个字符串常量，它包含了函数的实现代码。在函数中，可以使用 `$$` 字符串常量来定义 `function_body`。
```sql
create function find_film_by_id(
   id int
) returns film 
language sql
as 
$$
  select * from film 
  where film_id = id;  
$$; 
```

## 在存储过程中使用 $$ 字符串常量
```sql
create procedure proc_name(param_list)
language lang_name
as $$
  -- stored procedure body
$$
```

## Block structure

块语法
```sql
[ <<label>> ]
[ declare
    declarations ]
begin
    statements;
	...
end [ label ];
```

每个块包括两部分：
- 声明部分（declare）：声明变量、常量、游标等；
- 执行部分（begin...end）：执行语句。

执行部分可以嵌套。
```sql
do
$$
<<outer>>
declare
   x int = 0;
begin
   x = x + 1;
   <<inner>>
   declare
       y int = 2;
   begin
   	   y = y + x;
	   raise notice 'x=% y=%', x, y;
   end inner;
end outer;
$$
```